In [1]:
# import packages
import pandas as pd
import numpy as np
import sys
import re

sys.path.append("../tool/")

import preprocess

In [2]:
# load data
df2017 = pd.read_csv("../data/OriginalData/developer_survey_2017/survey_results_public_2017.csv")
df2018 = pd.read_csv("../data/OriginalData/developer_survey_2018/survey_results_public.csv")

/Users/renrui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# choose the fields
columns = [
    "Professional",
    "University",
    "EmploymentStatus",
    "FormalEducation",
    "MajorUndergrad",
    "Gender",
    "Race",
    "Country",
    "Salary",
    "Currency",
    "CompanySize",
    "DeveloperType",
    "JobSatisfaction",
    "JobSeekingStatus",
]

# field with single value
add_columns = {
    "SalaryType": "Yearly"
}

# rename the fields mapper
rename_columns = {
    "EmploymentStatus":"Employment",
    "MajorUndergrad" : "UndergradMajor",
    
}

In [4]:
# get the dataframe contained the fields choosed
df2017_certain = df2017.filter(items=columns, axis=1).copy()

# add the new fild SalaryType into the dataframe
for key, value in add_columns.items():
    df2017_certain[key] = value
    
# rename the columns name
df2017_certain.rename(rename_columns, axis=1, inplace=True)

In [5]:
df2017_certain.head()

,Professional,University,Employment,FormalEducation,UndergradMajor,Gender,Race,Country,Salary,Currency,CompanySize,DeveloperType,JobSatisfaction,JobSeekingStatus,SalaryType
0,Student,No,"Not employed, and not looking for work",Secondary school,NaN,Male,White or of European descent,United States,NaN,NaN,NaN,NaN,NaN,"I'm not actively looking, but I am open to new...",Yearly
1,Student,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,Male,White or of European descent,United Kingdom,NaN,British pounds sterling (£),20 to 99 employees,NaN,NaN,NaN,Yearly
2,Professional developer,No,Employed full-time,Bachelor's degree,Computer science or software engineering,Male,White or of European descent,United Kingdom,113750.0,British pounds sterling (£),"10,000 or more employees",Other,9.0,NaN,Yearly
3,Professional non-developer who sometimes write...,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,Male,White or of European descent,United States,NaN,NaN,"10,000 or more employees",NaN,3.0,I am actively looking for a job,Yearly
4,Professional developer,No,Employed full-time,Master's degree,Computer science or software engineering,NaN,NaN,Switzerland,NaN,NaN,10 to 19 employees,Mobile developer; Graphics programming; Deskto...,8.0,NaN,Yearly


In [6]:
# convert the element that is not Male or Female into NoComment
df2017_certain.Gender = df2017_certain.Gender.apply(preprocess.convert_single_func, 
                                                    args=(["Male", "Female"], 
                                                         "NoComment"))
# convert the Race no information value into NoInfo
# regrex pattern
pattern = re.compile(r"I don’t know|I prefer not to say")
df2017_certain.Race = df2017_certain.Race.apply(preprocess.convert_single_func, args=(pattern, "NoInfo", True))

# extract the value in the front of the first ;
df2017_certain.Race = df2017_certain.Race.str.split(";").apply(preprocess.convert_list_funct)


# extract the currecy value that is a alphabet value without a sign
pattern = re.compile(r"(\w*.*\s+\w*)+")
df2017_certain.Currency = df2017_certain.Currency.apply(preprocess.convert_single_func, args=(pattern, False, True))

In [7]:
# convert the COmpanySize no information value into NoInfo
pattern = re.compile(r"(I don't know)|(I prefer not to answer)")
df2017_certain.CompanySize = df2017_certain.CompanySize.apply(preprocess.convert_single_func, args=(pattern, "NoInfo", True))

In [8]:
# convert value about the JobSatisfaction field
def convert_job(x):
    # convert the x value into a string value according by the int value
    
    if x == 0:
        return "Extremely dissatisfied"
    elif x <= 2:
        return "Moderately dissatisfied"
    elif x <= 4:
        return "Slightly dissatisfied"
    elif x == 5:
        return "Neither satisfied nor dissatisfied"
    elif x <= 7:
        return "Slightly satisfied"
    elif x <= 9:
        return "Moderately satisfied"
    elif x == 10:
        return "Extremely satisfied"
    else:
        return x

df2017_certain.JobSatisfaction = df2017_certain.JobSatisfaction.apply(convert_job)

In [9]:
df2017_certain[~(df2017_certain.DeveloperType.str.find(";")>0)].DeveloperType.value_counts()

Web developer                                            10683
Mobile developer                                          1556
Desktop applications developer                            1433
Other                                                     1174
Embedded applications/devices developer                    664
Developer with a statistics or mathematics background      485
Data scientist                                             249
DevOps specialist                                          192
Quality assurance engineer                                 102
Database administrator                                      73
Graphics programming                                        55
Machine learning specialist                                 44
Systems administrator                                       31
Graphic designer                                             6
Name: DeveloperType, dtype: int64

In [10]:
df2018[~(df2018.DevType.str.find(";")>0)].DevType.value_counts()

Back-end developer                               6417
Full-stack developer                             6104
Mobile developer                                 3518
Student                                          3222
Front-end developer                              2608
Desktop or enterprise applications developer      896
Data scientist or machine learning specialist     869
Embedded applications or devices developer        708
QA or test developer                              552
Data or business analyst                          499
C-suite executive (CEO, CTO, etc.)                438
DevOps specialist                                 374
Engineering manager                               367
Educator or academic researcher                   351
System administrator                              305
Game or graphics developer                        220
Designer                                          168
Product manager                                   167
Database administrator      

In [11]:
raise

RuntimeError: No active exception to reraise

In [12]:
df2017_certain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Data columns (total 15 columns):
Professional        51392 non-null object
University          51392 non-null object
Employment          51392 non-null object
FormalEducation     51392 non-null object
UndergradMajor      42841 non-null object
Gender              35047 non-null object
Race                33033 non-null object
Country             51392 non-null object
Salary              12891 non-null float64
Currency            21408 non-null object
CompanySize         38922 non-null object
DeveloperType       36125 non-null object
JobSatisfaction     40376 non-null object
JobSeekingStatus    33380 non-null object
SalaryType          51392 non-null object
dtypes: float64(1), object(14)
memory usage: 5.9+ MB


In [13]:
# store the data
df2017_certain.to_csv("../data/survey2017.csv", index=False)